# 8. DataLoader(데이터로더)
***

## 8.1 Batch(배치)

- Manual data feed(데이터 피드 매뉴얼)

```python
# 0. 기초 데이터 불러오기

xy = np.loadtxtx('data-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = Variable(torch.from_numpy(xy[:, 0:-1]))
y_data = Variable(torch.from_numpy(xy[:, [-1]]))

# 1. Training Loop(반복 훈련 실시)

for epoch in range(100):
    # Forward pass: Compute predicted y by passing x to the model 
    y_pred = model(x_data)
    
    # Compute and print loss
    loss = criterion(y_pred, y_data)
    print(epoch, loss.data[0])
    
    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
```

- Batch(Batch size, 배치): The number of training examples in one forward/backward pass.
    - 한번 pass를 돌 때 훈련시킬 예시들의 수 
    - 모델 훈련시 아래의 코드와 같이 배치를 설정
    
    ```python
    # Training cycle
   for epoch in rnange(training_epochs):
          # Loop over all bathces   
          for i in range(total_batch):
              batch_xs, batch_ys = ...
    ```
    
- 신경망 용어 정리
    - epoch(에포크): 모든 훈련셋이 Forward Pass와 Backward Pass를 돌고오는 횟수
    - batch size(배치 사이즈): 한번의 전파(One pass)에 훈련되는 데이터셋의 수. 더 클수록 더 많은 메모리 공간을 필요로함
    - iterations(반복): 정해진 배치 사이즈를 이용하여 전파하는 횟수. One pass의 횟수(Foward pass + Backward pass = One pass)

## 8.2 DataLoader(데이터로더)

- DataLoader(데이터로더): 데이터를 다루기 위해 이용
- Original Data -> Queue -> iterable -> batch 순으로 이동 
```python
for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data # 기초 데이터 가져오기
    
    # wrap them in Variable
    inputs, labels = Variable(inputs), Variable(labels)
    
    # Run your training process
    print(epoch, i, "inputs", inputs.data, "labels", labels.data)
```

 ![5_Dataloader](slides/이미지/5_Dataloader.jpg)

- Custom DataLoader(데이터로더 커스텀)

```python
class DiabetesDataset(Dataset):
    """ Diabetes dataset."""
    
    # Initialize your data, download, etc.
    def __init__(self): # 1. Download, read data, etc.
        xy = np.loadtxt('data-diabetes.csv', delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        
    def __getitem__(self, index): # 2. Return one item on the index
        return self.x_data[index], self.y_data[index]
    
    def __len__(self): # 3. Return the data length 
        return self.len
    
dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32, # 배치사이즈 설정
                          shuffle=True,  # 큐에 올리기 전 기초데이터 셔플
                          num_workers=2)
```

데이터로딩 후 이어서 모델링 작업 실시하면 됨

- MNIST dataset lpading(MNIST 데이터셋 불러오기)

```python
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input PipeLine)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
```

## 8.3 Exercise
#### 1. Check out existing data sets(torch.vision)

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable

print(torchvision.datasets)

TypeError: 'module' object is not subscriptable

#### 2. Build DataLoader for 'Titanic'

In [12]:
from torch.utils.data import Dataset, DataLoader
from torch import from_numpy, tensor
import numpy as np

class DiabetesDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        xy = np.loadtxt('C:\\Users\\dbtmd\\OneDrive\\바탕 화면\\2_Pytorch Workspace\\2. Pytorch ZtoA\\diabetes.csv',
                        delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = from_numpy(xy[:, 0:-1])
        self.y_data = from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = tensor(inputs), tensor(labels)

        # Run your training process
        print(f'Epoch: {i} | Inputs {inputs.data} | Labels {labels.data}')

RuntimeError: DataLoader worker (pid(s) 15788, 15984) exited unexpectedly

#### 3. Build a classifier using the DataLoader